<a href="https://colab.research.google.com/github/AmitS1009/Using_BERT/blob/main/Using_PretrainedModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade transformers torch --quiet


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 85.2 MB/s eta 0:00:00


In [ ]:
from transformers import BertTokenizer, BertForMaskedLM

import torch
import torch.nn.functional as F

In [ ]:
#load tokenizer + pretrained BERT (MLM)

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForMaskedLM.from_pretrained('bert-base-uncased')

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
# input : (using [Mask] where we need to predict our token)
text = "The capital of France is [MASK]."


In [ ]:
#preprocess : tokenize+convert to tensors
inputs = tokenizer(text, return_tensors="pt")

In [ ]:
# forward to pass : no  gradient needed for inference
  #shape : [batch, seq_len, vocab]
with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits

In [ ]:
# finding the position of the [Mask] token :
mask_idx = (inputs["input_ids"] == tokenizer.mask_token_id).nonzero(as_tuple=True)[1]

In [ ]:
# softmax over vocab + take top-k prediction for that position

probs = F.softmax(logits[0, mask_idx], dim=-1)
top_k = torch.topk(probs, k=5, dim=-1)


In [ ]:
print("Top 5 Predictions for [MASK] :")
for token_id, prob in zip(top_k.indices[0], top_k.values[0]):
    token = tokenizer.decode([token_id])
    print(f"{token:<12} -> {prob.item():.4f}")


Top 5 Predictions for [MASK] :
paris        -> 0.4168
lille        -> 0.0714
lyon         -> 0.0634
marseille    -> 0.0444
tours        -> 0.0303


# Using PipeLine

In [ ]:
from transformers import pipeline, AutoModelForSequenceClassification, AutoTokenizer

model_name = "textattack/bert-base-uncased-SST-2"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

classifier = pipeline("sentiment-analysis", model = model, tokenizer = tokenizer)

result = classifier("The movie was masterpiece. I loved it!")

if result == 1:
  print("positive")
else :
  print("negative")

Device set to use cpu


negative
